# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
host = '127.0.0.1'

from cassandra.cluster import Cluster
cluster = Cluster([host])

# To establish connection and begin executing queries, need a session
try:
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace
create_ksp = """
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {
        'class': 'SimpleStrategy', 'replication_factor': 1
    };
"""
try:
    session.execute(create_ksp)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [12]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
'''
    - Choosing session_id and item_in_session as primary key and partition key, 
    since we are going to read data base on these columns and because we can distribute data
    in more number of nodes, improving query performance and avoiding node unbalance.

'''
query_drop_session_songs = "DROP TABLE IF EXISTS session_songs;"

query_create_session_songs = """

    CREATE TABLE IF NOT EXISTS session_songs (
        session_id int, 
        item_in_session int, 
        artist text, 
        song_title text, 
        length float,
        PRIMARY KEY((session_id, item_in_session))
    );
    
"""

try:
    session.execute(query_drop_session_songs)
    session.execute(query_create_session_songs)
except Exception as e:
    print(e)

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_songs (session_id, item_in_session, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s);"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query1 = """

    SELECT 
        * 
    FROM 
        session_songs 
    WHERE 
        session_id = 338 
        AND item_in_session = 4;
        
"""

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist, row.song_title, row.length)

338 4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query_drop_user_session_songs = "DROP TABLE IF EXISTS user_session_songs;"

query_create_user_session_songs = """

    CREATE TABLE IF NOT EXISTS user_session_songs (
        user_id int,
        session_id int,
        item_in_session int,
        artist text, 
        song_title text, 
        user_first_name text, 
        user_last_name text, 
        PRIMARY KEY((user_id, session_id), item_in_session)
    )
        WITH CLUSTERING ORDER BY (item_in_session DESC);
        
"""

try:
    session.execute(query_drop_user_session_songs)
    session.execute(query_create_user_session_songs)
except Exception as e:
    print(e)                    

In [17]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_songs (user_id, session_id, item_in_session, artist, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

In [20]:
query2 = """

    SELECT 
          artist
        , song_title
        , user_first_name
        , user_last_name 
    FROM 
         user_session_songs 
    WHERE 
        user_id = 10 
        AND session_id = 182;
        
"""

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.user_first_name, row.user_last_name)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


In [21]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query_drop_user_songs = """DROP TABLE IF EXISTS user_songs;"""

query_create_users_songs = """

    CREATE TABLE IF NOT EXISTS user_songs (
        song_title text,
        user_id int,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY ((song_title), user_id)
    );
    
"""

try:
    session.execute(query_drop_user_songs)
    session.execute(query_create_users_songs)
except Exception as e:
    print(e)

In [23]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s);"
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

In [24]:
query3 = """

    SELECT 
          user_first_name
        , user_last_name 
    FROM 
        user_songs 
    WHERE 
        song_title = 'All Hands Against His Own';
        
"""

try:
     rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [25]:
## TO-DO: Drop the table before closing out the sessions
try:
    session.execute(query_drop_session_songs)
    session.execute(query_drop_user_session_songs)
    session.execute(query_drop_user_songs)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()